# Train Pixel Level Annotation Model

### This notebook uses pixel level annotations to train a Random Forest Classifier to predict labels

We assume Pixel level annotations are available, as produced by the "../data/annotations/transform_polygon_annotations_to_pixels.ipynb" notebook. Feature selection and grid_search for optimal parameters has been done in a separate notebook ("Coepelduynen/make_train_model_on_annotations_coepelduynen.ipynb") and those outcomes are taken as given in this notebook.

Change the set Variables cell below as desired and then run the entire notebook to get cross_validation results as well as a final model trained on all data.

Date: 2024-01-12\
Author: Pieter Kouyzer, Michael de Winter

In [ ]:
%load_ext autoreload
%autoreload 2

In [6]:
%matplotlib notebook
import pandas as pd
import geopandas as gpd
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
import pprint
import os
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from satellite_images_nso_datascience.training.train import train_imbalanced_model, cross_validation_balance_on_date
from satellite_images_nso_datascience.training.utils import get_cross_validation_results_filepath, get_model_filepath
from satellite_images_nso_datascience.training.metric_calculation import calculate_average_metrics, get_metrics
from sklearn.metrics import f1_score
from satellite_images_nso_datascience.other import functions
import satellite_images_nso_datascience.model_metrics.custom_model_metrics as custom_model_metrics
i

data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..')).replace("\\","/")+"/data/annotations/annotations_pixel_dataframes/"

2024/08/14 12:55:53 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


In [ ]:
# Set Variables
location = "Nieuwkoopse_plassen"
satellite_constellation = "PNEO"

if location == "Voornes Duin":

    if satellite_constellation == "PNEO":
        # Check if file exists
        if not os.path.isfile(data_dir+"Voornes_Duin_PNEO_2024-01-29_pixel_annotations.parquet"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/Voornes_Duin/annotations_pixel_dataframes/2023_PNEO/Voornes_Duin_PNEO_2024-01-29_pixel_annotations.parquet", data_dir+"Voornes_Duin_PNEO_2024-01-29_pixel_annotations.parquet")

        annotated_pixels_filepath = data_dir+"Voornes_Duin_PNEO_2024-01-29_pixel_annotations.parquet"

    #TODO: Missing Superview here!


elif location == "Coepelduynen":
    if satellite_constellation == "PNEO":
        # Check if file exists
        if not os.path.isfile(data_dir+"annotaties_coepelduynen_to_pixel_2023.parquet"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/coepelduynen/annotations_pixel_dataframes/annotations_2023_PNEO/annotaties_coepelduynen_to_pixel_2023.parquet", data_dir+"annotaties_coepelduynen_to_pixel_2023.parquet")

        annotated_pixels_filepath = data_dir+"annotaties_coepelduynen_to_pixel_2023.parquet"
        df = pd.read_parquet(annotated_pixels_filepath)

    if satellite_constellation == "Superview":
        # Check if file exists
        if not os.path.isfile(data_dir+"annotaties_coepelduynen_to_pixel.csv"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/coepelduynen/annotations_pixel_dataframes/annotations_2019_2022_Superview/annotaties_coepelduynen_to_pixel.csv", data_dir+"annotaties_coepelduynen_to_pixel.csv")

        annotated_pixels_filepath = data_dir+"annotaties_coepelduynen_to_pixel.csv"
        df = pd.read_parquet(annotated_pixels_filepath)

   
elif location == "Schippersgat":
    if satellite_constellation == "PNEO":
        if not os.path.isfile(data_dir+"PNEO_waterplanten_annotations.parquet"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/Schippersgat/PNEO_waterplanten_annotations.parquet", data_dir+"PNEO_waterplanten_annotations.parquet")
        annotated_pixels_filepath = data_dir+"PNEO_waterplanten_annotations_schippersgat.parquet"
        df = pd.read_parquet(annotated_pixels_filepath)

    if satellite_constellation == "Superview":
        if not os.path.isfile(data_dir+"Superview_waterplanten_annotations.parquet"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/Schippersgat/Superview_waterplanten_annotations.parquet", data_dir+"Superview_waterplanten_annotations.parquet")
        annotated_pixels_filepath = data_dir+"Superview_waterplanten_annotations.parquet"
        df = pd.read_parquet(annotated_pixels_filepath)

elif location == "Nieuwkoopse_plassen":
    if satellite_constellation == "PNEO":
        if not os.path.isfile(data_dir+"PNEO_waterplanten_annotations.parquet"):
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/Schippersgat/PNEO_waterplanten_annotations.parquet", data_dir+"PNEO_waterplanten_annotations.parquet")
        annotated_pixels_filepath = data_dir+"PNEO_waterplanten_annotations_schippersgat.parquet"
        df = pd.read_parquet(annotated_pixels_filepath)


        df = pd.concat([df, pd.read_parquet(data_dir+"PNEO_waterplanten_annotations_Nieuwkoopse_Plassen.parquet")])

    if satellite_constellation == "Superview":
        
        if not os.path.isfile(data_dir+"Superview_waterplanten_annotations.parquet"):
            print("Downloading file")
            functions.download_file("https://e34a505986aa74678a5a0e0f.blob.core.windows.net/satellite-images-nso/Nieuwkoopse_Plassen/Superview_waterplanten_annotations.parquet", data_dir+"Superview_waterplanten_annotations.parquet")
        annotated_pixels_filepath = data_dir+"Superview_waterplanten_annotations.parquet"
        df = pd.read_parquet(annotated_pixels_filepath)

        #TODO: VERY STRANGE REASON THAT NDWI AND NDVI GET FLIPPED WHILE READING IN DATA!!!!!!!!!!!!!!!!!!!!
        df = df.rename(columns={'ndvi': 'ndwi', 'ndwi': 'ndvi'})

In [ ]:
# Optimal parameters and features Voornes
if location == "Voornes Duin":
    selected_features = ['r', 'g', 'b', 'n', 'e', 'd', 'ndvi','re_ndvi', 'height']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "max_features": "auto", 
        "bootstrap": False
    }

elif location == "Coepelduynen":
    # Optimal parameters and features
    selected_features = ['r','g','b','n','e','d','ndvi','re_ndvi']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "bootstrap": False
    }

elif location == "Schippersgat" and satellite_constellation == "Superview":
    # Optimal parameters and features
    selected_features = ['r', 'g', 'b', 'i', 'ndvi', 'height']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "bootstrap": False
    }

elif location == "Schippersgat" and satellite_constellation == "PNEO":
    # Optimal parameters and features
    selected_features = ['r', 'g', 'b', 'n', 'e', 'd','ndvi', 're_ndvi']
    optimal_parameters = {
        "n_estimators": 10, 
        "min_samples_split": 5, 
        "min_samples_leaf": 1,
        "bootstrap": False
    }


elif location == "Nieuwkoopse_plassen" and satellite_constellation == "PNEO":
    # Optimal parameters and features
    selected_features = ['r', 'g', 'b', 'n', 'e', 'd','ndvi', 're_ndvi']
    optimal_parameters = {
        "n_estimators": 1, 
        "min_samples_split": 2, 
        "min_samples_leaf": 1,
        "bootstrap": False
    }

elif location == "Nieuwkoopse_plassen" and satellite_constellation == "Superview":
    # Optimal parameters and features
    selected_features = ['r', 'g', 'b', 'i', 'ndvi', 'ndwi']
    optimal_parameters = {
        "n_estimators": 1, 
        "min_samples_split": 2, 
        "min_samples_leaf": 1,
        "bootstrap": True
    }

In [ ]:
# This is to give an indication of the amount of data points per label
df['label'].value_counts()

In [ ]:
# Experiment to test if downsampling of Water is necessary.
#df = pd.concat([df[df["label"] == "Water"].sample(1719330, random_state=1), df[df["label"] == "Ground"], df[df["label"] == "Waterplants"]])

### Cross Validation

We do cross-validation, where the folds are decided by the 'date' column. This is to avoid pixels from the same image from ending up in both the train and test datasets. We display the metrics averaged over the folds and write the results to a pickle.

In [ ]:
model = RandomForestClassifier(**optimal_parameters)
scaler = StandardScaler()

In [ ]:
if location == "Voornes Duin":
    cv =5 
    random_state = 1337
    sampling_type_boundary = 100000
elif location == "Coepelduynen":
    cv =4 
    random_state = 1337
    sampling_type_boundary = 898609
elif location == "Schippersgat":
    cv =2
    random_state = 1337
    sampling_type_boundary = 898609

elif location == "Nieuwkoopse_plassen":
    cv = 7
    random_state = 1337
    sampling_type_boundary = 898609

In [ ]:
df['date'] = df['date'].str.split('_').str[0]

In [ ]:
results = cross_validation_balance_on_date(data=df, model=model, cv=cv, features=selected_features, random_state=random_state, sampling_type_boundary=sampling_type_boundary , scaler=scaler)

In [ ]:
calculate_average_metrics(results=results)

In [ ]:
cross_validation_results_filepath = get_cross_validation_results_filepath(location=location, satellite_constellation=satellite_constellation, df=df)
print(f"Saving to {cross_validation_results_filepath}")
with open(cross_validation_results_filepath, "wb") as file:
    pickle.dump(results, file)

## Cross validation randomised

In [ ]:
# Initialize KFold
kf = KFold(n_splits=4, shuffle=True, random_state=42)

scores = cross_val_score(model, df[selected_features], df["label"], cv=kf)

In [ ]:
scores

### Try some unbalanced models

In [ ]:
unbalanced_model = RandomForestClassifier(**optimal_parameters)
unblanced_final_scaler = StandardScaler()

In [ ]:
unbalanced_model.fit(unblanced_final_scaler.fit_transform(df[selected_features]), df['label'])

In [ ]:
f1_score(unbalanced_model.predict(unblanced_final_scaler.transform(df[selected_features])), df["label"], average=None)

In [ ]:
final_artefact = {
    "model": unbalanced_model,
    "scaler": unblanced_final_scaler
}

### Export Definitive model.

Trains a Random Forest Classifier model on all data and writes it to a pickle file for later use. This is the definitive model output by this notebook.

In [ ]:
final_model = RandomForestClassifier(**optimal_parameters)
final_scaler = StandardScaler()

train_imbalanced_model(
    X_train=df[selected_features], 
    y_train=df["label"], 
    model=final_model, 
    random_state=1337, 
    sampling_type_boundary=sampling_type_boundary ,
    scaler=final_scaler
)
pprint.pprint(get_metrics(y=df["label"], X=df[selected_features], model=final_model, scaler=final_scaler))

In [ ]:
final_artefact = {
    "model": final_model,
    "scaler": final_scaler
}

In [ ]:
final_model_filepath =get_model_filepath(location=location, satellite_constellation=satellite_constellation, df=df).replace(".sav","_test.sav").replace("..","C:/repos/satellite-images-nso-datascience/")
print(f"Saving to {final_model_filepath}")
with open(final_model_filepath, "wb") as file:
    pickle.dump(final_artefact, file)

# Custom Model Metrics



In [ ]:
final_artefact = pickle.load(open(final_model_filepath, "rb"))
nk_plassen_metrics = custom_model_metrics.custom_model_metrics(final_artefact['model'], final_artefact['scaler'], "Nieuwkoopse_plassen", "Superview")

In [5]:
nk_plassen_metrics.metric_difficult_pixels()

'Error'

# NDWI and NDVI do not match, somehow gets twisted, do not forget this!

In [ ]:
df[(df["rd_y"]  == 463906.25) & (df["rd_x"] == 117219.25)][selected_features]

In [ ]:
df_annotated_export_data[(df_annotated_export_data["rd_y"]  == 463906.25) & (df_annotated_export_data["rd_x"] == 117219.25)][selected_features]

In [ ]:
final_model.predict(final_scaler.transform(df_annotated_export_data[(df_annotated_export_data["rd_y"]  == 463906.25) & (df_annotated_export_data["rd_x"] == 117219.25)][selected_features]))